# Intro to Deep Learning Systems : HW 2
## Problem 1: Perceptron

In [1]:
# Imports
import numpy as np
from numpy.random import default_rng
import pandas as pd

In [2]:
# Generate training data
def generate_data(n):
    x = default_rng().uniform(low=0.0, high=1.0, size=(n, 2))
    func = lambda x: 1 if x else -1
    y = np.vectorize(func)(x[:, 0] > x[:, 1])
    
    return x, y
X_train, y_train = generate_data(10)
print("Training Data")
np.concatenate((X_train, y_train.reshape(-1, 1)), axis=1)

Training Data


array([[ 0.77630199,  0.67073049,  1.        ],
       [ 0.54433576,  0.95879851, -1.        ],
       [ 0.03631605,  0.11214203, -1.        ],
       [ 0.16496294,  0.00253814,  1.        ],
       [ 0.73906551,  0.88131042, -1.        ],
       [ 0.48309446,  0.53066175, -1.        ],
       [ 0.7533649 ,  0.74075462,  1.        ],
       [ 0.48779942,  0.79262143, -1.        ],
       [ 0.85317393,  0.99522347, -1.        ],
       [ 0.22314569,  0.09628791,  1.        ]])

In [3]:
def train_perceptron(X_train, y_train, epoch_lim=1000, hinge_loss=False):
    """Train weights for given data via perceptron algorithm.

    Args:
        X_train: Feature matrix
        y_train: Class labels
        epoch_lim: Upper epoch limit to cut-off training
        hinge_loss: Specify whether to use hinge loss

    Returns:
        Trained weights
    """
    w = np.zeros(X_train.shape[1])
    epoch, convergence = 0, False
    threshold = 1 if hinge_loss else 0

    while not convergence and epoch < epoch_lim:
        convergence = True
        for i in range(len(X_train)):
            x = X_train[i]
            y = y_train[i]

            margin = y * np.dot(w, x)
            if margin <= threshold:
                convergence = False
                w += y * x

        epoch += 1
    
    if not convergence:
        print("Convergence not achieved. Too many epochs.")
    return w

def predict_perceptron(x, weights):
    """Predict values using perceptron weights

    Args:
        x: Input features
        weights: Perceptron weights

    Return:
        Output vector
    """

    return np.sign(np.dot(x, weights))

In [4]:
# Training
w_perceptron = train_perceptron(X_train, y_train)
w_hinge = train_perceptron(X_train, y_train, hinge_loss=True)

# Test
perceptron_accuracy, hinge_accuracy = [], []
for i in range(10):
    X_test, y_test = generate_data(5000)
    pred_perceptron = predict_perceptron(X_test, w_perceptron)
    pred_hinge = predict_perceptron(X_test, w_hinge)
    
    perceptron_accuracy.append(sum(pred_perceptron == y_test) / pred_perceptron.shape[0])
    hinge_accuracy.append(sum(pred_hinge == y_test) / pred_hinge.shape[0])

print(f"Mean accuracy with perceptron loss: {np.mean(perceptron_accuracy)}")
print(f"Mean accuracy with hinge loss: {np.mean(hinge_accuracy)}")

Mean accuracy with perceptron loss: 0.96868
Mean accuracy with hinge loss: 0.98608


## Problem 2: Weight Initialization, Dead Neurons, Leaky ReLU

In [5]:
from deepreplay.callbacks import ReplayData
from deepreplay.replay import Replay
from deepreplay.plot import compose_plots
from keras.initializers import normal
from matplotlib import pyplot as plt
from q2.model_builder import *

In [6]:
from deepreplay.datasets.ball import load_data

X, y = load_data(n_dims=10)

In [12]:
filename = 'part2_weight_initializers.h5'
group_name = 'sigmoid_stdev_0.01'

# Uses normal initializer
initializer = normal(mean=0, stddev=0.01, seed=13)

# Builds BLOCK model
model = build_model(n_layers=5, input_dim=10, units=100, 
                    activation='sigmoid', initializer=initializer)

# Since we only need initial weights, we don't even need to train the model! 
# We still use the ReplayData callback, but we can pass the model as argument instead
replaydata = ReplayData(X, y, filename=filename, group_name=group_name, model=model)

# Now we feed the data to the actual Replay object
# so we can build the visualizations
replay = Replay(replay_filename=filename, group_name=group_name)

# Using subplot2grid to assemble a complex figure...
fig = plt.figure(figsize=(12, 6))
ax_zvalues = plt.subplot2grid((2, 2), (0, 0))
ax_weights = plt.subplot2grid((2, 2), (0, 1))
ax_activations = plt.subplot2grid((2, 2), (1, 0))
ax_gradients = plt.subplot2grid((2, 2), (1, 1))

wv = replay.build_weights(ax_weights)
gv = replay.build_gradients(ax_gradients)
# Z-values
zv = replay.build_outputs(ax_zvalues, before_activation=True, 
                          exclude_outputs=True, include_inputs=False)
# Activations
av = replay.build_outputs(ax_activations, exclude_outputs=True, include_inputs=False)

# Finally, we use compose_plots to update all
# visualizations at once
fig = compose_plots([zv, wv, av, gv], 
                    epoch=0, 
                    title=r'Activation: sigmoid - Initializer: Normal $\sigma = 0.01$')

ValueError: Unable to synchronously create dataset (name already exists)